#Hello Hello#

In [21]:
!pip install pandas numpy scikit-learn Sastrawi

In [22]:
import pandas as pd
import numpy as np
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.preprocessing import MinMaxScaler


In [23]:
df = pd.read_csv('Merged.csv')
print(f"Ukuran dataset: {df.shape}")
print("\nContoh 5 data pertama:")
display(df.head())

Ukuran dataset: (10670, 10)

Contoh 5 data pertama:


,Judul,Harga,Lokasi,Kamar,Wc,Parkir,Luas_tanah,Luas_bangunan,Deskripsi,Keywords
0,Di Jual Harga Rendah Rumah Luas Di Daerah Duwet,Rp 800 Juta,"Wonosari, Gunung Kidul",3.0,3.0,1,408 m²,205 m²,DI JUAL HARGA RENDAH RUMAH LUAS DI DAERAH DUWE...,Rumah\nFull Furnished\nSiap Huni\nDekat Sekolah
1,Rumah Murah Full Furnish Di Patuk Dekat Heha S...,Rp 320 Juta,"Patuk, Gunung Kidul",2.0,1.0,1,70 m²,36 m²,"RUMAH PATUK Lokasi : Patuk, Gunung Kidul, Daer...",Rumah\nFull Furnished
2,Rumah Baru di Playen Gunung Kidul Dekat RSUD W...,Rp 400 Juta,"Playen, Gunung Kidul",2.0,1.0,1,80 m²,50 m²,NaN,Rumah\nDekat Fasilitas Kesehatan
3,Rumah Cantik Full Furnished Hanya 300JTAN Di P...,Rp 320 Juta,"Patuk, Gunung Kidul",2.0,1.0,1,70 m²,36 m²,RUMAH CANTIK FULL FURNISHED HANYA 300JTAN DI P...,Rumah\nFull Furnished\nDekat Sekolah
4,Rumah Scandinavian 200Jt an di Karangmojo Gunu...,Rp 280 Juta,"Karangmojo, Gunung Kidul",2.0,1.0,1,100 m²,40 m²,Promo Launching Perdana 200Juta-an! Tersedia K...,Rumah\nDekat Sekolah\nDekat Fasilitas Kesehata...


In [24]:
print("\nJumlah missing values per kolom:")
print(df.isnull().sum())


Jumlah missing values per kolom:
Judul              2
Harga              2
Lokasi             2
Kamar            105
Wc               117
Parkir             0
Luas_tanah         4
Luas_bangunan    115
Deskripsi         35
Keywords           2
dtype: int64


In [25]:
def clean_price(price_str):
    if pd.isna(price_str):
        return np.nan
    numbers = re.findall(r'[\d,]+', str(price_str))
    if not numbers:
        return np.nan
    number = float(numbers[0].replace(',', ''))
    if 'miliar' in price_str.lower() or 'milyar' in price_str.lower():
        return number * 1000000000
    elif 'juta' in price_str.lower():
        return number * 1000000
    return number

def clean_area(area_str):
    if pd.isna(area_str):
        return np.nan
    numbers = re.findall(r'[\d.]+', str(area_str))
    return float(numbers[0]) if numbers else np.nan

In [26]:
df['Harga_Clean'] = df['Harga'].apply(clean_price)
df['Luas_tanah_Clean'] = df['Luas_tanah'].apply(clean_area)
df['Luas_bangunan_Clean'] = df['Luas_bangunan'].apply(clean_area)

In [27]:
stemmer = StemmerFactory().create_stemmer()
stopword_remover = StopWordRemoverFactory().create_stop_word_remover()

In [28]:
def clean_text(text):
    if pd.isna(text):
        return ""
    # Lowercase
    text = str(text).lower()
    # Hapus karakter khusus tapi pertahankan koma dan titik
    text = re.sub(r'[^\w\s,.]', ' ', text)
    # Hapus multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Hapus stopwords
    text = stopword_remover.remove(text)
    # Stemming
    text = stemmer.stem(text)
    return text

In [30]:
text_columns = ['Judul', 'Deskripsi', 'Keywords', 'Lokasi']
for col in text_columns:
    df[f'{col}_Clean'] = df[col].apply(clean_text)

# 4. Normalisasi data numerik
numeric_columns = ['Harga_Clean', 'Luas_tanah_Clean', 'Luas_bangunan_Clean', 'Kamar', 'Wc', 'Parkir']

# Handle missing values sebelum normalisasi
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())

# Normalisasi menggunakan MinMaxScaler
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df[numeric_columns])

# Tambahkan hasil normalisasi ke dataframe
normalized_columns = [col + '_Normalized' for col in numeric_columns]
for idx, col in enumerate(normalized_columns):
    df[col] = normalized_data[:, idx]

In [45]:
final_df = pd.DataFrame({
    # Data original
    'Judul': df['Judul'],
    'Lokasi': df['Lokasi'],

    # Data numerik original
    'Harga': df['Harga_Clean'],
    'Kamar': df['Kamar'],
    'WC': df['Wc'],
    'Parkir': df['Parkir'],
    'Luas_Tanah': df['Luas_tanah_Clean'],
    'Luas_Bangunan': df['Luas_bangunan_Clean'],

    # Data numerik yang sudah dinormalisasi
    'Harga_Normalized': df['Harga_Clean_Normalized'],
    'Kamar_Normalized': df['Kamar_Normalized'],
    'WC_Normalized': df['Wc_Normalized'],
    'Parkir_Normalized': df['Parkir_Normalized'],
    'Luas_Tanah_Normalized': df['Luas_tanah_Clean_Normalized'],
    'Luas_Bangunan_Normalized': df['Luas_bangunan_Clean_Normalized'],

    # Data teks yang sudah dipreprocess
    'Judul_Clean': df['Judul_Clean'],
    'Lokasi_Clean': df['Lokasi_Clean'],
    'Deskripsi_Clean': df['Deskripsi_Clean'],
    'Keywords_Clean': df['Keywords_Clean']
})

In [48]:
final_df = final_df.dropna()
final_df = final_df.drop_duplicates()

In [49]:
print(final_df.isnull().sum())

Judul                       0
Lokasi                      0
Harga                       0
Kamar                       0
WC                          0
Parkir                      0
Luas_Tanah                  0
Luas_Bangunan               0
Harga_Normalized            0
Kamar_Normalized            0
WC_Normalized               0
Parkir_Normalized           0
Luas_Tanah_Normalized       0
Luas_Bangunan_Normalized    0
Judul_Clean                 0
Lokasi_Clean                0
Deskripsi_Clean             0
Keywords_Clean              0
dtype: int64


In [50]:
final_df.to_csv("processed_house_data.csv", index=False)
print("\nData telah diproses dan disimpan. Contoh hasil processing:")
display(final_df.head())


Data telah diproses dan disimpan. Contoh hasil processing:


,Judul,Lokasi,Harga,Kamar,WC,Parkir,Luas_Tanah,Luas_Bangunan,Harga_Normalized,Kamar_Normalized,WC_Normalized,Parkir_Normalized,Luas_Tanah_Normalized,Luas_Bangunan_Normalized,Judul_Clean,Lokasi_Clean,Deskripsi_Clean,Keywords_Clean
0,Di Jual Harga Rendah Rumah Luas Di Daerah Duwet,"Wonosari, Gunung Kidul",800000000.0,3.0,3.0,1,408.0,205.0,0.000766,0.025316,0.010638,0.0,0.000116,0.000174,jual harga rendah rumah luas daerah duwet,wonosari gunung kidul,jual harga rendah rumah luas daerah duwet wono...,rumah full furnished siap huni dekat sekolah
1,Rumah Murah Full Furnish Di Patuk Dekat Heha S...,"Patuk, Gunung Kidul",320000000.0,2.0,1.0,1,70.0,36.0,0.000285,0.012658,0.000000,0.0,0.000020,0.000030,rumah murah full furnish patuk dekat heha sky ...,patuk gunung kidul,rumah patuk lokasi patuk gunung kidul daerah i...,rumah full furnished
2,Rumah Baru di Playen Gunung Kidul Dekat RSUD W...,"Playen, Gunung Kidul",400000000.0,2.0,1.0,1,80.0,50.0,0.000365,0.012658,0.000000,0.0,0.000023,0.000042,rumah baru playen gunung kidul dekat rsud wono...,playen gunung kidul,,rumah dekat fasilitas sehat
3,Rumah Cantik Full Furnished Hanya 300JTAN Di P...,"Patuk, Gunung Kidul",320000000.0,2.0,1.0,1,70.0,36.0,0.000285,0.012658,0.000000,0.0,0.000020,0.000030,rumah cantik full furnished 300jtan patuk,patuk gunung kidul,rumah cantik full furnished 300jtan patuk ruma...,rumah full furnished dekat sekolah
4,Rumah Scandinavian 200Jt an di Karangmojo Gunu...,"Karangmojo, Gunung Kidul",280000000.0,2.0,1.0,1,100.0,40.0,0.000245,0.012658,0.000000,0.0,0.000028,0.000033,rumah scandinavian 200jt an karangmojo gunungk...,karangmojo gunung kidul,promo launching perdana 200juta an sedia kavli...,rumah dekat sekolah dekat fasilitas sehat deka...


In [34]:
print("\nStatistik data numerik yang sudah dinormalisasi:")
display(final_df[[col for col in final_df.columns if 'Normalized' in col]].describe())


Statistik data numerik yang sudah dinormalisasi:


,Harga_Normalized,Kamar_Normalized,WC_Normalized,Parkir_Normalized,Luas_Tanah_Normalized,Luas_Bangunan_Normalized
count,10670.000000,10670.000000,10670.000000,10670.000000,10670.000000,10670.000000
mean,0.047725,0.033809,0.008725,0.027243,0.002006,0.000688
std,0.095938,0.044990,0.019880,0.059715,0.040102,0.013966
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000501,0.012658,0.000000,0.000000,0.000025,0.000040
50%,0.000926,0.025316,0.005319,0.000000,0.000033,0.000079
75%,0.037003,0.037975,0.010638,0.052632,0.000049,0.000136
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
df2 = pd.read_csv('processed_house_data.csv')
print(f"Ukuran dataset: {df2.shape}")
print("\nContoh 5 data pertama:")
display(df2.head())

Ukuran dataset: (7603, 18)

Contoh 5 data pertama:


,Judul,Lokasi,Harga,Kamar,WC,Parkir,Luas_Tanah,Luas_Bangunan,Harga_Normalized,Kamar_Normalized,WC_Normalized,Parkir_Normalized,Luas_Tanah_Normalized,Luas_Bangunan_Normalized,Judul_Clean,Lokasi_Clean,Deskripsi_Clean,Keywords_Clean
0,Di Jual Harga Rendah Rumah Luas Di Daerah Duwet,"Wonosari, Gunung Kidul",800000000.0,3.0,3.0,1,408.0,205.0,0.000766,0.025316,0.010638,0.0,0.000116,0.000174,jual harga rendah rumah luas daerah duwet,wonosari gunung kidul,jual harga rendah rumah luas daerah duwet wono...,rumah full furnished siap huni dekat sekolah
1,Rumah Murah Full Furnish Di Patuk Dekat Heha S...,"Patuk, Gunung Kidul",320000000.0,2.0,1.0,1,70.0,36.0,0.000285,0.012658,0.000000,0.0,0.000020,0.000030,rumah murah full furnish patuk dekat heha sky ...,patuk gunung kidul,rumah patuk lokasi patuk gunung kidul daerah i...,rumah full furnished
2,Rumah Baru di Playen Gunung Kidul Dekat RSUD W...,"Playen, Gunung Kidul",400000000.0,2.0,1.0,1,80.0,50.0,0.000365,0.012658,0.000000,0.0,0.000023,0.000042,rumah baru playen gunung kidul dekat rsud wono...,playen gunung kidul,NaN,rumah dekat fasilitas sehat
3,Rumah Cantik Full Furnished Hanya 300JTAN Di P...,"Patuk, Gunung Kidul",320000000.0,2.0,1.0,1,70.0,36.0,0.000285,0.012658,0.000000,0.0,0.000020,0.000030,rumah cantik full furnished 300jtan patuk,patuk gunung kidul,rumah cantik full furnished 300jtan patuk ruma...,rumah full furnished dekat sekolah
4,Rumah Scandinavian 200Jt an di Karangmojo Gunu...,"Karangmojo, Gunung Kidul",280000000.0,2.0,1.0,1,100.0,40.0,0.000245,0.012658,0.000000,0.0,0.000028,0.000033,rumah scandinavian 200jt an karangmojo gunungk...,karangmojo gunung kidul,promo launching perdana 200juta an sedia kavli...,rumah dekat sekolah dekat fasilitas sehat deka...
